In [1]:
import netCDF4
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
from numba import njit, typed, types

from rasterio.mask import mask
from rasterio.io import DatasetReader,DatasetWriter

import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
os.chdir('/home/jovyan/data/aghanghas/inundation_map/outputs')


rem='/home/jovyan/data/aghanghas/inundation_map/hand_src/010802hand.tif'
catchments='/home/jovyan/data/aghanghas/inundation_map/hand_src/010802catchhuc.tif'

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


In [2]:
comid_df=pd.read_csv('/home/jovyan/data/aghanghas/inundation_map/hand_src/010802_comid.txt', sep=" ", usecols=[0], header=None)
comid=comid_df[0].values.tolist()

In [3]:
def main_trib_index(comid):
    nwm_folder='/home/jovyan/data/aghanghas/inundation_map/nwm-v2-1/'
    f_name="201211071600.CHRTOUT_DOMAIN1.comp"
    f_path=nwm_folder+f_name
    data=netCDF4.Dataset(f_path)  # type: ignore
    COMID_list=data['feature_id'][:].data
    main_index=np.searchsorted(COMID_list,comid)
    
    data.close()
    return (main_index)

comid_index= main_trib_index(comid)

In [4]:
nwm_folder='/home/jovyan/data/aghanghas/inundation_map/nwm-v2-1/'
f_name="201211071600.CHRTOUT_DOMAIN1.comp"
f_path=nwm_folder+f_name
nwm_dis_all=netCDF4.Dataset(f_path)
comid_dis=nwm_dis_all['streamflow'][comid_index].data
src_table=pd.read_csv('/home/jovyan/data/aghanghas/inundation_map/hand_src/hydrogeo-fulltable-010802.csv', sep=",", usecols=['CatchId','Stage','Discharge (m3s-1)'])

In [5]:
stage_dis_df=pd.DataFrame(index=comid,columns=['discharge','stage'])
stage_dis_df['discharge']=comid_dis

In [6]:
for id in comid:
    dis=stage_dis_df['discharge'][id]
    sub_src_table=src_table[src_table['CatchId']==id]
    interpolated_stage = np.interp(dis,sub_src_table.loc[:,'Discharge (m3s-1)'],sub_src_table.loc[:,'Stage'])
    h = round(interpolated_stage,4)
    h = types.float32(h)
    stage_dis_df['stage'][id]=h
    

/tmp/ipykernel_622/2490915904.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stage_dis_df['stage'][id]=h


In [7]:
rem = rasterio.open(rem)
catchments = rasterio.open(catchments)

In [8]:
out_shape=rem.read(1).shape

In [9]:
a1=rem.read(1).flatten()
b1=catchments.read(1).flatten()
indundation_array=a1.copy()

In [23]:
for id in comid:
    stage=stage_dis_df['stage'][id]
    if len(a1[b1==id]) >0:
        if a1[b1==id].max()>0:
            depth=stage - abs(a1[b1==id])
            a1[b1==id]=depth
a1.reshape(a1,out_shape)

KeyboardInterrupt: 